# Foundations of Computer Science - progetto 2022/23

- Mattia Birti [897092] 
- Alberto Porrini [826306]
- Gloria Longo [864579]

You have to work on the Dogs adoptions dataset:
*  [Dogs](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogs.csv)
*  [Dog Travel](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogTravel.csv)
*  [NST-EST2021-POP](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/NST-EST2021-POP.csv)

### Notes

1. It is mandatory to use GitHub for developing the project.
1. The project must be a jupyter notebook.
1. There is no restriction on the libraries that can be used, nor on the Python version.
1. All questions on the project must be asked in a public channel on Zulip.
1. At most 3 students can be in each group. You must create the groups by yourself.
1. You do not have to send me the project before the discussion.s

## Libraries

In [1]:
import pandas as pd

### Import data from GitHub

#### Dogs.csv

In [2]:
dogs = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogs.csv", sep=",", encoding='latin-1')

In [3]:
dogs.shape

(58180, 37)

In [4]:
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


#### DogTravel.csv

In [5]:
dogTravel = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogTravel.csv", sep=",", encoding='latin-1')

In [6]:
dogTravel.shape

(6194, 9)

In [7]:
dogTravel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


#### NST-EST.csv

In [8]:
nstest = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/NST-EST2021-POP.csv", header=None, sep=",", encoding='latin-1')

In [9]:
nstest.shape

(51, 2)

In [10]:
nstest.head()

,0,1
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223


## 1. Extract all dogs with status that is not adoptable

## 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

Let's see all the primary breed with a groupBy:

In [11]:
nBreed = dogs.groupby('breed_primary').count()[['species','breed_secondary']]
nBreed

,species,breed_secondary
breed_primary,,
Affenpinscher,17,2
Afghan Hound,4,1
Airedale Terrier,19,9
Akbash,3,0
Akita,181,52
...,...,...
Wirehaired Pointing Griffon,1,0
Wirehaired Terrier,60,18
Xoloitzcuintli / Mexican Hairless,11,4


As we can see we can use "species" as total number of individues for this species.
Affenpischer are in total 17 dogs, 2 of those are mixed breed.

Rename colums

In [12]:
nBreed = nBreed.rename( columns = {'species':'total', 'breed_secondary':'nSecondary'})
nBreed

,total,nSecondary
breed_primary,,
Affenpinscher,17,2
Afghan Hound,4,1
Airedale Terrier,19,9
Akbash,3,0
Akita,181,52
...,...,...
Wirehaired Pointing Griffon,1,0
Wirehaired Terrier,60,18
Xoloitzcuintli / Mexican Hairless,11,4


We have to compute how many are primary breed:

In [13]:
nBreed['nPrimary'] = nBreed['total'] - nBreed['nSecondary']
nBreed

,total,nSecondary,nPrimary
breed_primary,,,
Affenpinscher,17,2,15
Afghan Hound,4,1,3
Airedale Terrier,19,9,10
Akbash,3,0,3
Akita,181,52,129
...,...,...,...
Wirehaired Pointing Griffon,1,0,1
Wirehaired Terrier,60,18,42
Xoloitzcuintli / Mexican Hairless,11,4,7


It's time to compute the ratio between Primary and Secondary Breed.

In [14]:
nBreed['ratioBreed'] = nBreed['nPrimary'] / nBreed['nSecondary']
nBreed

,total,nSecondary,nPrimary,ratioBreed
breed_primary,,,,
Affenpinscher,17,2,15,7.500000
Afghan Hound,4,1,3,3.000000
Airedale Terrier,19,9,10,1.111111
Akbash,3,0,3,inf
Akita,181,52,129,2.480769
...,...,...,...,...
Wirehaired Pointing Griffon,1,0,1,inf
Wirehaired Terrier,60,18,42,2.333333
Xoloitzcuintli / Mexican Hairless,11,4,7,1.750000


As we can see, there are many 'inf' values. It is correct because for that breed there are not any secondary breed.